# Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

## Loading Data

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [3]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32)

## Defining Model
2 convolutional Layer, 2 Dense layer

In [4]:
class CNN(nn.Module):
    def __init__(self,dropout):
        super().__init__()
        self.dropout=dropout
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv1=nn.Conv2d(1,64,kernel_size=3)
        self.conv2=nn.Conv2d(64,64,kernel_size=3)
        self.dense1=nn.Linear(64*5*5,256)
        self.dense2=nn.Linear(256,10)
        
        self.dropout=nn.Dropout(0.1)
        
    def forward(self,images):
        shape=images.size()
        temp=self.conv1(images)
        temp=self.pool(temp)
        temp=F.relu(temp)
        temp=self.conv2(temp)
        temp=self.pool(temp)
        temp=F.relu(temp)
        temp=temp.view(shape[0],-1)
        temp=self.dense1(temp)
        temp=F.relu(temp)
        if self.dropout:
            temp=self.dropout(temp)
        temp=self.dense2(temp)
        return temp

In [5]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

### We are using Cross entropy as our loss function

In [6]:
criterion = nn.CrossEntropyLoss(reduction='sum').to(device)
epochs=100

all_losses=[]

# Using Optimizer SGD, Adam, RMSprop with and without regularization and Dropout
### 12 such cases are below:

## SGD without Regularization and without Dropout

In [7]:
model=CNN(dropout=False).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    
    
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("SGD.png",dpi=500)
plt.clf()

  1%|▌                                                                              | 6/938 [00:00<00:18, 50.01it/s]

Epoch = 1	Sum of Training loss = 72368.37129974365	Correct Validation Examples = 8753


  1%|▌                                                                              | 6/938 [00:00<00:16, 55.55it/s]

Epoch = 2	Sum of Training loss = 20957.367738246918	Correct Validation Examples = 9258


  1%|▍                                                                              | 5/938 [00:00<00:19, 48.09it/s]

...

  1%|▊                                                                              | 9/938 [00:00<00:10, 85.94it/s]

Epoch = 99	Sum of Training loss = 476.5848149652593	Correct Validation Examples = 9900


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 268.91it/s]


Epoch = 100	Sum of Training loss = 494.11150859622285	Correct Validation Examples = 9901


<Figure size 1152x360 with 0 Axes>

In [8]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 99.11


## SGD without Regularization and with Dropout

In [9]:
model=CNN(dropout=True).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("SGD_dropout.png",dpi=500)
plt.clf()

  2%|█▋                                                                            | 20/938 [00:00<00:09, 96.52it/s]

Epoch = 1	Sum of Training loss = 74766.29827022552	Correct Validation Examples = 8945


  1%|▊                                                                             | 10/938 [00:00<00:10, 91.99it/s]

Epoch = 2	Sum of Training loss = 20481.24946975708	Correct Validation Examples = 9275


  1%|▊                                                                              | 9/938 [00:00<00:10, 89.35it/s]

...

  1%|▊                                                                             | 10/938 [00:00<00:09, 93.71it/s]

Epoch = 99	Sum of Training loss = 445.7056709395256	Correct Validation Examples = 9897


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 283.75it/s]


Epoch = 100	Sum of Training loss = 486.71295703342184	Correct Validation Examples = 9904


<Figure size 1152x360 with 0 Axes>

In [10]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 99.06


## SGD with Regularization and without Dropout

In [11]:
model=CNN(dropout=False).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4, weight_decay=0.2)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("SGD_regularization.png",dpi=500)
plt.clf()

  1%|▊                                                                             | 10/938 [00:00<00:09, 98.26it/s]

Epoch = 1	Sum of Training loss = 74565.70112991333	Correct Validation Examples = 8746


  2%|█▋                                                                            | 20/938 [00:00<00:09, 95.67it/s]

Epoch = 2	Sum of Training loss = 21724.75870847702	Correct Validation Examples = 9203


  1%|▊                                                                             | 10/938 [00:00<00:09, 92.85it/s]

...

  1%|▉                                                                            | 12/938 [00:00<00:08, 112.45it/s]

Epoch = 99	Sum of Training loss = 2228.1894810199738	Correct Validation Examples = 9870


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:00<00:00, 322.54it/s]


Epoch = 100	Sum of Training loss = 2219.0919147729874	Correct Validation Examples = 9881


<Figure size 1152x360 with 0 Axes>

In [12]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 98.91


## SGD with Regularization and with Dropout

In [13]:
model=CNN(dropout=True).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4,weight_decay=0.2)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("SGD_regularization_droput.png",dpi=500)
plt.clf()

  2%|█▉                                                                           | 23/938 [00:00<00:08, 109.93it/s]

Epoch = 1	Sum of Training loss = 78269.18190860748	Correct Validation Examples = 8864


  2%|█▊                                                                           | 22/938 [00:00<00:08, 108.74it/s]

Epoch = 2	Sum of Training loss = 21559.877737522125	Correct Validation Examples = 9227


  1%|▊                                                                             | 10/938 [00:00<00:09, 95.49it/s]

...

  1%|▉                                                                            | 11/938 [00:00<00:08, 107.08it/s]

Epoch = 99	Sum of Training loss = 2243.774956971407	Correct Validation Examples = 9887


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 298.30it/s]


Epoch = 100	Sum of Training loss = 2229.0767874121666	Correct Validation Examples = 9894


<Figure size 1152x360 with 0 Axes>

In [14]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 98.96000000000001


## Adam without Regularization and without Dropout

In [15]:
model=CNN(dropout=False).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("Adam.png",dpi=500)
plt.clf()

  2%|█▋                                                                           | 21/938 [00:00<00:08, 102.12it/s]

Epoch = 1	Sum of Training loss = 28637.39568924904	Correct Validation Examples = 9577


  1%|▊                                                                             | 10/938 [00:00<00:09, 99.68it/s]

Epoch = 2	Sum of Training loss = 7552.2219178676605	Correct Validation Examples = 9760


  1%|▉                                                                            | 11/938 [00:00<00:09, 101.49it/s]

...

  2%|█▊                                                                           | 22/938 [00:00<00:08, 105.87it/s]

Epoch = 99	Sum of Training loss = 27.11791973605	Correct Validation Examples = 9922


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 304.85it/s]


Epoch = 100	Sum of Training loss = 28.23759792058948	Correct Validation Examples = 9920


<Figure size 1152x360 with 0 Axes>

In [16]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 99.32


## Adam without Regularization and with Dropout

In [17]:
model=CNN(dropout=True).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("Adam_dropout.png",dpi=500)
plt.clf()

  1%|▉                                                                            | 11/938 [00:00<00:08, 107.05it/s]

Epoch = 1	Sum of Training loss = 26918.20462346077	Correct Validation Examples = 9579


  1%|▉                                                                            | 11/938 [00:00<00:09, 102.59it/s]

Epoch = 2	Sum of Training loss = 7369.3410539627075	Correct Validation Examples = 9754


  1%|▉                                                                            | 11/938 [00:00<00:08, 104.05it/s]

...

  1%|▉                                                                            | 11/938 [00:00<00:08, 108.13it/s]

Epoch = 99	Sum of Training loss = 45.67445508418128	Correct Validation Examples = 9925


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:00<00:00, 319.58it/s]


Epoch = 100	Sum of Training loss = 11.072530914075742	Correct Validation Examples = 9922


<Figure size 1152x360 with 0 Axes>

In [18]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 99.32


## Adam with Regularization and without Dropout

In [19]:
model=CNN(dropout=False).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.2)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("Adam_Regularization.png",dpi=500)
plt.clf()

  1%|▉                                                                            | 11/938 [00:00<00:08, 105.04it/s]

Epoch = 1	Sum of Training loss = 26936.256699323654	Correct Validation Examples = 9608


  1%|▉                                                                            | 11/938 [00:00<00:08, 105.04it/s]

Epoch = 2	Sum of Training loss = 7922.296985268593	Correct Validation Examples = 9724


  1%|▉                                                                            | 11/938 [00:00<00:08, 107.11it/s]

...

  1%|▉                                                                            | 11/938 [00:00<00:08, 105.04it/s]

Epoch = 99	Sum of Training loss = 2308.0375143438578	Correct Validation Examples = 9886


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:00<00:00, 317.64it/s]


Epoch = 100	Sum of Training loss = 2298.8401795625687	Correct Validation Examples = 9860


<Figure size 1152x360 with 0 Axes>

In [20]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 99.00999999999999


## Adam with Regularization and with Dropout

In [21]:
model=CNN(dropout=True).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.2)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("Adam_Regularization_dropout.png",dpi=500)
plt.clf()

  1%|▉                                                                            | 11/938 [00:00<00:08, 103.08it/s]

Epoch = 1	Sum of Training loss = 27813.681656360626	Correct Validation Examples = 9537


  1%|▉                                                                            | 11/938 [00:00<00:08, 104.05it/s]

Epoch = 2	Sum of Training loss = 8669.060554742813	Correct Validation Examples = 9713


  1%|▉                                                                            | 11/938 [00:00<00:09, 101.19it/s]

...

  1%|▉                                                                            | 12/938 [00:00<00:08, 111.41it/s]

Epoch = 99	Sum of Training loss = 2305.273868754506	Correct Validation Examples = 9874


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 303.47it/s]


Epoch = 100	Sum of Training loss = 2325.406769245863	Correct Validation Examples = 9886


<Figure size 1152x360 with 0 Axes>

In [22]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 98.95


## RMSprop without Regularization and without Dropout

In [23]:
model=CNN(dropout=False).to(device)
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("RMS.png",dpi=500)
plt.clf()

  1%|▉                                                                            | 11/938 [00:00<00:08, 105.04it/s]

Epoch = 1	Sum of Training loss = 19690.01674413681	Correct Validation Examples = 9643


  1%|▉                                                                            | 12/938 [00:00<00:08, 113.51it/s]

Epoch = 2	Sum of Training loss = 6779.742224574089	Correct Validation Examples = 9768


  1%|▉                                                                            | 11/938 [00:00<00:08, 104.05it/s]

...

  1%|▉                                                                            | 11/938 [00:00<00:08, 105.04it/s]

Epoch = 99	Sum of Training loss = 16.276006559186385	Correct Validation Examples = 9923


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 298.04it/s]


Epoch = 100	Sum of Training loss = 15.39421605805714	Correct Validation Examples = 9930


<Figure size 1152x360 with 0 Axes>

In [24]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 99.37


## RMSprop without Regularization and with Dropout

In [25]:
model=CNN(dropout=True).to(device)
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("RMS_dropout.png",dpi=500)
plt.clf()

  1%|▉                                                                            | 11/938 [00:00<00:08, 105.02it/s]

Epoch = 1	Sum of Training loss = 19597.465118408203	Correct Validation Examples = 9646


  1%|▉                                                                            | 11/938 [00:00<00:08, 106.05it/s]

Epoch = 2	Sum of Training loss = 6865.240772843361	Correct Validation Examples = 9781


  1%|▊                                                                             | 10/938 [00:00<00:09, 93.71it/s]

...

  2%|█▊                                                                           | 22/938 [00:00<00:08, 105.99it/s]

Epoch = 99	Sum of Training loss = 23.208746710423483	Correct Validation Examples = 9925


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 310.72it/s]


Epoch = 100	Sum of Training loss = 26.90147629815018	Correct Validation Examples = 9919


<Figure size 1152x360 with 0 Axes>

In [26]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 99.27


## RMSprop with Regularization and without Dropout

In [27]:
model=CNN(dropout=False).to(device)
optimizer = optim.RMSprop(model.parameters(), lr=1e-4, weight_decay=0.2)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("RMS_regularization.png",dpi=500)
plt.clf()

  1%|▉                                                                            | 11/938 [00:00<00:09, 101.19it/s]

Epoch = 1	Sum of Training loss = 21194.69036912918	Correct Validation Examples = 9603


  2%|█▊                                                                           | 22/938 [00:00<00:08, 107.25it/s]

Epoch = 2	Sum of Training loss = 7885.934823870659	Correct Validation Examples = 9729


  1%|▉                                                                            | 11/938 [00:00<00:08, 109.20it/s]

...

  1%|▉                                                                            | 11/938 [00:00<00:08, 104.05it/s]

Epoch = 99	Sum of Training loss = 2402.7782043665648	Correct Validation Examples = 9890


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 303.75it/s]


Epoch = 100	Sum of Training loss = 2354.808225721121	Correct Validation Examples = 9882


<Figure size 1152x360 with 0 Axes>

In [28]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 98.99


## RMSprop with Regularization and with Dropout

In [29]:
model=CNN(dropout=True).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4, weight_decay=0.2)
train_loss_array=[]
valid_acc_array=[]
for epoch in range(epochs):
    #Traning 
    train_loss=0
    for data in tqdm(trainloader):
        images,labels=data
        images=images.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss_array.append(train_loss)
    #validation
    correct=0
    model.eval()
    with torch.no_grad():
        for data in tqdm(testloader):
            images,labels=data
            images=images.to(device)
            labels=labels.to(device)
            output=model(images)
            _,pred=torch.max(output.data,1)
            correct+=(pred == labels).sum().item()
    valid_acc_array.append(correct)
    model.train()
    print(f"Epoch = {epoch+1}\tSum of Training loss = {train_loss}\tCorrect Validation Examples = {correct}")
    
fig,ax=plt.subplots(1,2,figsize=(16,5))
ax[0].plot(np.arange(1,epochs+1),train_loss_array)
ax[0].set(ylabel="Summed Training loss",xlabel="Epochs")
ax[1].plot(np.arange(1,epochs+1),np.array(valid_acc_array)/len(testset)*100)
ax[1].set(ylabel="Validation Accuracy",xlabel="Epochs")
plt.savefig("RMS_regularization_dropout.png",dpi=500)
plt.clf()

  1%|▊                                                                             | 10/938 [00:00<00:09, 98.30it/s]

Epoch = 1	Sum of Training loss = 79961.41120147705	Correct Validation Examples = 8817


  1%|▉                                                                            | 12/938 [00:00<00:08, 109.37it/s]

Epoch = 2	Sum of Training loss = 21825.286368370056	Correct Validation Examples = 9235


  1%|▉                                                                            | 11/938 [00:00<00:08, 104.05it/s]

...

  1%|▉                                                                            | 11/938 [00:00<00:09, 102.12it/s]

Epoch = 99	Sum of Training loss = 2228.2651437669992	Correct Validation Examples = 9900


100%|████████████████████████████████████████████████████████████████████████████| 313/313 [00:01<00:00, 309.72it/s]


Epoch = 100	Sum of Training loss = 2214.875375151634	Correct Validation Examples = 9886


<Figure size 1152x360 with 0 Axes>

In [30]:
print(f'Maximum Accuracy achieved = {max(valid_acc_array)/len(testset)*100}')
all_losses.append(train_loss_array)

Maximum Accuracy achieved = 99.0


In [31]:
labels=['SGD','Adam','RMSprop']
title=['Without Dropout or Regulaization', 'With Dropout only', 'With Regularization only', 'With Droput and Regularization']
for i in range(4):
    fig,ax=plt.subplots(1,1,figsize=(16,10))
    for j in range(3):
        ax.plot(np.arange(1,epochs+1),all_losses[4*j+i],label=labels[j])
        ax.set(xlabel='Epochs',ylabel='Summed Loss')
    plt.legend()
    plt.title(title[i])
    plt.savefig(f"{title[i]}.png")
    plt.clf()

<Figure size 1152x720 with 0 Axes>

<Figure size 1152x720 with 0 Axes>

<Figure size 1152x720 with 0 Axes>

<Figure size 1152x720 with 0 Axes>